# **Лабораторная работа №3. Random Forest.**

In [ ]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/bank-full.csv', decimal = ',', sep = ';')
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [ ]:
# Считаю, сколько пропусков в каждом столбце

NAs = pd.concat([df.isnull().sum()], axis=1)
NAs[NAs.sum(axis=1) > 0]

,0


In [ ]:
df.y.value_counts()

no     39922
yes     5289
Name: y, dtype: int64

In [ ]:
df['default'] = np.where(df['default'] == 'yes', 1, 0)
df['housing'] = np.where(df['housing'] == 'yes', 1, 0)
df['loan'] = np.where(df['loan'] == 'yes', 1, 0)
df['y'] = np.where(df['y'] == 'yes', 1, 0)
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,0,2143,1,0,unknown,5,may,261,1,-1,0,unknown,0
1,44,technician,single,secondary,0,29,1,0,unknown,5,may,151,1,-1,0,unknown,0
2,33,entrepreneur,married,secondary,0,2,1,1,unknown,5,may,76,1,-1,0,unknown,0
3,47,blue-collar,married,unknown,0,1506,1,0,unknown,5,may,92,1,-1,0,unknown,0
4,33,unknown,single,unknown,0,1,0,0,unknown,5,may,198,1,-1,0,unknown,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,0,825,0,0,cellular,17,nov,977,3,-1,0,unknown,1
45207,71,retired,divorced,primary,0,1729,0,0,cellular,17,nov,456,2,-1,0,unknown,1
45208,72,retired,married,secondary,0,5715,0,0,cellular,17,nov,1127,5,184,3,success,1
45209,57,blue-collar,married,secondary,0,668,0,0,telephone,17,nov,508,4,-1,0,unknown,0


In [ ]:
df.dtypes

age           int64
job          object
marital      object
education    object
default       int64
balance       int64
housing       int64
loan          int64
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y             int64
dtype: object

Считаю, что колонки 'contact', 'day' и 'month' не являются информативными, поэтому удаляю их:

In [ ]:
df.drop(['contact', 'day', 'month'], axis=1, inplace = True)
df

,age,job,marital,education,default,balance,housing,loan,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,0,2143,1,0,261,1,-1,0,unknown,0
1,44,technician,single,secondary,0,29,1,0,151,1,-1,0,unknown,0
2,33,entrepreneur,married,secondary,0,2,1,1,76,1,-1,0,unknown,0
3,47,blue-collar,married,unknown,0,1506,1,0,92,1,-1,0,unknown,0
4,33,unknown,single,unknown,0,1,0,0,198,1,-1,0,unknown,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,0,825,0,0,977,3,-1,0,unknown,1
45207,71,retired,divorced,primary,0,1729,0,0,456,2,-1,0,unknown,1
45208,72,retired,married,secondary,0,5715,0,0,1127,5,184,3,success,1
45209,57,blue-collar,married,secondary,0,668,0,0,508,4,-1,0,unknown,0


In [ ]:
# преобразую категориальные признаки с помощью метода Get Dummies

df = pd.get_dummies(df, columns = ['job', 'marital', 'education', 'poutcome'], drop_first= True)
df.head()

,age,default,balance,housing,loan,duration,campaign,pdays,previous,y,...,job_unemployed,job_unknown,marital_married,marital_single,education_secondary,education_tertiary,education_unknown,poutcome_other,poutcome_success,poutcome_unknown
0,58,0,2143,1,0,261,1,-1,0,0,...,0,0,1,0,0,1,0,0,0,1
1,44,0,29,1,0,151,1,-1,0,0,...,0,0,0,1,1,0,0,0,0,1
2,33,0,2,1,1,76,1,-1,0,0,...,0,0,1,0,1,0,0,0,0,1
3,47,0,1506,1,0,92,1,-1,0,0,...,0,0,1,0,0,0,1,0,0,1
4,33,0,1,0,0,198,1,-1,0,0,...,0,1,0,1,0,0,1,0,0,1


In [ ]:
#  разделяю предикоторы и отклик

X = df.iloc[:, df.columns != 'y'].values
y = df.iloc[:, 9].values

In [ ]:
#  разделяю на обучающую и тестовую выборки

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state = 100)

**Балансирую выборку с помощью библиотеки SMOTE**

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smote = SMOTE()

X_smote, y_smote = smote.fit_resample(X_train, y_train)

print('Original dataset shape', len(X_train))
print('Resampled dataset shape', len(X_smote))

Original dataset shape 30291
Resampled dataset shape 53492


**Подбор внешних параметров с помощью GridSearch**

In [ ]:
param_grid = {
    'n_estimators': [30, 40, 50, 100],
    'max_depth': [4, 5, 6],
    'min_samples_split': [3, 5, 10],
  }

In [ ]:
model = RandomForestClassifier(random_state=12,
                               n_estimators=80,
                               criterion='gini',
                               max_depth=5,
                               max_features='auto',
                               max_leaf_nodes=None,
                               min_impurity_decrease=0.001,
                               bootstrap=True,
                               min_samples_leaf=10,
                               min_samples_split=15,
                               verbose=1,
                               oob_score=True,
                               warm_start=False,
                               class_weight=None)


grid_search_res = GridSearchCV(model, param_grid=param_grid, cv=2,
                               #  выбор критерия качества
                               scoring='accuracy',
                               #  поведение в случае ошибки при вычислении критерия качества
                               error_score='raise',
                               #  обучить модель с лучшими значениями параметроы
                               refit=True,
                               pre_dispatch=None, verbose=0)

In [ ]:
grid_search_res.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn

GridSearchCV(cv=2, error_score='raise',
             estimator=RandomForestClassifier(max_depth=5, max_features='auto',
                                              min_impurity_decrease=0.001,
                                              min_samples_leaf=10,
                                              min_samples_split=15,
                                              n_estimators=80, oob_score=True,
                                              random_state=12, verbose=1),
             param_grid={'max_depth': [4, 5, 6],
                         'min_samples_split': [3, 5, 10],
                         'n_estimators': [30, 40, 50, 100]},
             pre_dispatch=None, scoring='accuracy')

In [ ]:
#   Лучший набор значений гиперпараметров

best_hyperparams = grid_search_res.best_params_
print('Лучшие значения гиперпараметров:\n', best_hyperparams)

Лучшие значения гиперпараметров:
 {'max_depth': 6, 'min_samples_split': 3, 'n_estimators': 40}


In [ ]:
# Значение критерия качества для лучших значений гиперпараметров
# Mean cross-validated score of the best_estimator
best_CV_score = grid_search_res.best_score_
print('Лучшее accuracy', best_CV_score)

Лучшее accuracy 0.8831666115703488


In [ ]:
# Лучшая модель
best_model = grid_search_res.best_estimator_

# accuracy на тестовом множестве для лучшей модели
test_acc = best_model.score(X_test,y_test)

# Значение критерия качества для лучших значений гиперпараметров
print('Лучшее accuracy на тестовом множестве', test_acc)

Лучшее accuracy на тестовом множестве 0.8831099195710456


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.1s finished
